# نصب پکیج های مورد نیاز

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

# ایمپورت کتابخانه ها و توابع مورد نیاز

In [21]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [22]:
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import tensorflow as tf


# اتصال به پایگاه داده و گرفتن دیتا

In [24]:
import pyodbc

conn = pyodbc.connect(
        'DRIVER=ODBC Driver 17 for SQL Server;SERVER=GWH,1433;DATABASE=golestanml_test;UID=GC-Rezaie_ai;PWD=R3z@!e123;')

In [25]:
sql_clause = """SELECT
        dc.CustomerBk,
        f.Order_Date,
        f.Product_Code,
        dc.CityBk,
        dc.AreaBk,
        dc.CustomerTypeBk,
        dc.OfficeBk,
        ci.CityTitle,
        so.Actived,
        dc.LastCustomerClassBk,
        dc.DistPathCode
FROM [DWAI].[Sale].[FactV4] f (NOLOCK)
JOIN [DWAI].[Sale].[DimCustomer] dc ON f.CustomerCode = dc.CustomerBk
JOIN [DWAI].[regional].[DimSaleOffice] so ON dc.OfficeBk = so.SaleOfficeBk
JOIN [DWAI].[regional].[DimCity] ci on ci.CityBK = dc.CityBk
WHERE Order_Date BETWEEN 13980630 AND 14010330
AND f.CustomerCode IN (
            SELECT
                    dc.CustomerBk
            FROM [DWAI].[Sale].[DimCustomer] dc
            WHERE dc.DeactiveDate IS NULL
            AND dc.CustomerBk IN (
                    SELECT CustomerCode
                    FROM [DWAI].[Sale].[FactV4]
                )
)"""

a = pd.read_sql(sql_clause, conn)

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [26]:
a

,CustomerBk,Order_Date,Product_Code,CityBk,AreaBk,CustomerTypeBk,OfficeBk,CityTitle,Actived,LastCustomerClassBk,DistPathCode
0,972442240,13980904,218,437,0,37,325,ملك شهر,True,0,0
1,292034224,13980904,381,149,0,37,995,ايلام,True,100,1
2,959503144,13980904,275,112,0,37,959,تهران,True,300,0
3,959503144,13980904,278,112,0,37,959,تهران,True,300,0
4,970301938,13980924,207,547,0,37,970,قشم,True,-10,0
...,...,...,...,...,...,...,...,...,...,...,...
9257796,171060689,14000116,1571,1047,0,37,290,ماكو,True,0,8
9257797,608005438,14000116,1571,112,334,9,902,تهران,True,100,24
9257798,222442878,14000116,1576,128,0,37,159,يزد,True,300,0
9257799,220443923,14000116,1576,127,0,37,220,ياسوج,True,300,0


In [27]:
sql_clause = """SELECT
        dc.CustomerBk,
        f.Order_Date,
        f.Product_Code,
        dc.CityBk,
        dc.AreaBk,
        dc.CustomerTypeBk,
        dc.OfficeBk,
        ci.CityTitle,
        so.Actived,
        dc.LastCustomerClassBk,
        dc.DistPathCode
FROM [DWAI].[Sale].[FactV4] f (NOLOCK)
JOIN [DWAI].[Sale].[DimCustomer] dc ON f.CustomerCode = dc.CustomerBk
JOIN [DWAI].[regional].[DimSaleOffice] so ON dc.OfficeBk = so.SaleOfficeBk
JOIN [DWAI].[regional].[DimCity] ci on ci.CityBK = dc.CityBk
WHERE Order_Date BETWEEN 14010331 AND 14011030
AND f.CustomerCode IN (
            SELECT
                    dc.CustomerBk
            FROM [DWAI].[Sale].[DimCustomer] dc
            WHERE dc.DeactiveDate IS NULL
            AND dc.CustomerBk IN (
                    SELECT CustomerCode
                    FROM [DWAI].[Sale].[FactV4]
                )
)"""

b = pd.read_sql(sql_clause, conn)

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [28]:
b

,CustomerBk,Order_Date,Product_Code,CityBk,AreaBk,CustomerTypeBk,OfficeBk,CityTitle,Actived,LastCustomerClassBk,DistPathCode
0,142560827,14010529,1664,112,0,52,149,تهران,True,250,0
1,987083109,14010529,218,117,0,37,987,ساري,True,0,0
2,993083385,14010529,805,271,0,37,993,لاهيجان,True,300,0
3,970302304,14010917,2520,547,0,22,970,قشم,True,200,0
4,970302304,14010917,2521,547,0,22,970,قشم,True,200,0
...,...,...,...,...,...,...,...,...,...,...,...
2242998,191000233,14010614,496,1280,3,37,236,نيمور,True,50,2
2242999,959503035,14010614,643,112,0,37,959,تهران,True,300,0
2243000,999561785,14010615,1770,112,0,37,999,تهران,True,300,0
2243001,471083831,14010615,230,310,0,37,986,نوشهر,True,300,0


# دو برابر کردن دیتای 6 ماه اخیر

In [29]:
c=pd.concat([b, b])

# تجمیع دیتای دوبرابر شده 6 ماه قبل و داده های قبلی تا 3 سال قبل به میزان یک برابر

In [30]:
df=pd.concat([a, c])

In [31]:
df

,CustomerBk,Order_Date,Product_Code,CityBk,AreaBk,CustomerTypeBk,OfficeBk,CityTitle,Actived,LastCustomerClassBk,DistPathCode
0,972442240,13980904,218,437,0,37,325,ملك شهر,True,0,0
1,292034224,13980904,381,149,0,37,995,ايلام,True,100,1
2,959503144,13980904,275,112,0,37,959,تهران,True,300,0
3,959503144,13980904,278,112,0,37,959,تهران,True,300,0
4,970301938,13980924,207,547,0,37,970,قشم,True,-10,0
...,...,...,...,...,...,...,...,...,...,...,...
2242998,191000233,14010614,496,1280,3,37,236,نيمور,True,50,2
2242999,959503035,14010614,643,112,0,37,959,تهران,True,300,0
2243000,999561785,14010615,1770,112,0,37,999,تهران,True,300,0
2243001,471083831,14010615,230,310,0,37,986,نوشهر,True,300,0


In [32]:
df1=df.reset_index()

In [33]:
df=df1

In [28]:
df.to_csv('first1.csv',index=False)

In [134]:
df = pd.read_csv('first1.csv')

In [34]:
df

,index,CustomerBk,Order_Date,Product_Code,CityBk,AreaBk,CustomerTypeBk,OfficeBk,CityTitle,Actived,LastCustomerClassBk,DistPathCode
0,0,972442240,13980904,218,437,0,37,325,ملك شهر,True,0,0
1,1,292034224,13980904,381,149,0,37,995,ايلام,True,100,1
2,2,959503144,13980904,275,112,0,37,959,تهران,True,300,0
3,3,959503144,13980904,278,112,0,37,959,تهران,True,300,0
4,4,970301938,13980924,207,547,0,37,970,قشم,True,-10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13743802,2242998,191000233,14010614,496,1280,3,37,236,نيمور,True,50,2
13743803,2242999,959503035,14010614,643,112,0,37,959,تهران,True,300,0
13743804,2243000,999561785,14010615,1770,112,0,37,999,تهران,True,300,0
13743805,2243001,471083831,14010615,230,310,0,37,986,نوشهر,True,300,0


In [35]:
df=df[['CustomerBk','CustomerTypeBk','Product_Code','AreaBk','CityTitle','OfficeBk','DistPathCode','LastCustomerClassBk']]

In [36]:
len(df)

13743807

In [37]:
set(df['OfficeBk'][df['CityTitle']=='تبريز'])

{133, 279, 280, 327, 345, 346}

In [38]:
len(set(list(df['OfficeBk'][df['CityTitle']=='تبريز'])+list(df['OfficeBk'][df['CityTitle']=='تهران'])+list(df['OfficeBk'][df['CityTitle']=='اصفهان'])+list(df['OfficeBk'][df['CityTitle']=='مشهد'])))

40

In [39]:
aaa=list(set(list(df['OfficeBk'][df['CityTitle']=='تبريز'])+list(df['OfficeBk'][df['CityTitle']=='تهران'])+list(df['OfficeBk'][df['CityTitle']=='اصفهان'])+list(df['OfficeBk'][df['CityTitle']=='مشهد'])+list(df['OfficeBk'][df['CityTitle']=='شيراز'])+list(df['OfficeBk'][df['CityTitle']=='يزد'])+list(df['OfficeBk'][df['CityTitle']=='اراک'])+list(df['OfficeBk'][df['CityTitle']=='اهواز'])+list(df['OfficeBk'][df['CityTitle']=='رشت'])))

In [40]:
len(aaa)

55

In [41]:
aaa

[129,
 260,
 133,
 902,
 134,
 132,
 138,
 267,
 268,
 139,
 272,
 273,
 146,
 147,
 148,
 149,
 150,
 279,
 280,
 154,
 155,
 156,
 157,
 158,
 159,
 289,
 936,
 948,
 956,
 959,
 320,
 324,
 325,
 326,
 327,
 200,
 971,
 975,
 336,
 977,
 210,
 339,
 976,
 982,
 983,
 345,
 346,
 220,
 230,
 999,
 998,
 233,
 234,
 237,
 250]

In [42]:
# len(set(list(df['OfficeBk'][df['CityTitle']=='تهران'])))

In [43]:
# set(list(df['OfficeBk'][df['CityTitle']=='تهران']))

# تفکیک دیتای کلانشهرها از غیر کلانشهرها

In [44]:
lll=[]
lll1 = []
for i in range(len(df)):
    if df['OfficeBk'][i] in aaa:
        lll.append(i)  
    else:
        lll1.append(i)

In [45]:
len(df)

13743807

In [46]:
len(lll)

6542548

In [47]:
len(lll1)

7201259

# ادامه با دیتای کلانشهرها

In [48]:
df2=df.iloc[lll]

In [49]:
df3=df2.reset_index()

In [50]:
df4=df3.iloc[:,1:]

In [51]:
df4

,CustomerBk,CustomerTypeBk,Product_Code,AreaBk,CityTitle,OfficeBk,DistPathCode,LastCustomerClassBk
0,972442240,37,218,0,ملك شهر,325,0,0
1,959503144,37,275,0,تهران,959,0,300
2,959503144,37,278,0,تهران,959,0,300
3,621004521,37,1511,132,تهران,956,3,100
4,622501396,38,2864,151,تهران,146,2,300
...,...,...,...,...,...,...,...,...
6542543,611040281,37,1080,504,تهران,156,1,100
6542544,954560659,37,1298,0,تهران,250,0,300
6542545,619040248,37,1511,11,تهران,149,1,50
6542546,959503035,37,643,0,تهران,959,0,300


In [52]:
df = df4

In [53]:

df=df.dropna(axis=0, subset=['OfficeBk'])

In [54]:
df = df.reset_index()

In [55]:
df=df.iloc[:,1:]

In [56]:
len(df)

6542548

# خواندن لیست محصولات فعال

In [61]:
df1=pd.read_csv('product.csv')

In [62]:
f=df1['ProductBK'].unique()

In [63]:
f=list(f)

In [64]:
len(f)

543

# استخراج تراکنشهایی که در آنها محول فعالی خریداری شده

In [65]:
f1=[]
for i in range(len(df)):
    if df['Product_Code'][i] in f:
        f1.append(i)

In [66]:
len(df)

6542548

In [67]:
len(f1)

6441011

In [68]:
df2=df.iloc[f1]

In [69]:
df3=df2.reset_index()

In [70]:
df4=df3.iloc[:,1:]

In [71]:
df4

,CustomerBk,CustomerTypeBk,Product_Code,AreaBk,CityTitle,OfficeBk,DistPathCode,LastCustomerClassBk
0,972442240,37,218,0,ملك شهر,325,0,0
1,959503144,37,275,0,تهران,959,0,300
2,959503144,37,278,0,تهران,959,0,300
3,621004521,37,1511,132,تهران,956,3,100
4,622501396,38,2864,151,تهران,146,2,300
...,...,...,...,...,...,...,...,...
6441006,611040281,37,1080,504,تهران,156,1,100
6441007,954560659,37,1298,0,تهران,250,0,300
6441008,619040248,37,1511,11,تهران,149,1,50
6441009,959503035,37,643,0,تهران,959,0,300


In [72]:
df = df4

In [73]:
df['CityTitle'].value_counts()

تهران      2385073
مشهد        617416
رشت         280823
شيراز       261158
اهواز       253651
            ...   
هوراند           2
سنگان            2
كنگان            2
نيشابور          1
قم               1
Name: CityTitle, Length: 431, dtype: int64

In [74]:
df['DistPathCode'] = df['DistPathCode'].astype(str)

In [75]:
df['OfficeBk'] = df['OfficeBk'].astype(str)

In [76]:
df['LastCustomerClassBk'] = df['LastCustomerClassBk'].astype(str)

In [77]:
df['CustomerBk'] = df['CustomerBk'].astype(str)

In [78]:
df['CustomerTypeBk'] = df['CustomerTypeBk'].astype(str)

In [79]:
df['AreaBk'] = df['AreaBk'].astype(str)

In [80]:
df['Product_Code'] = df['Product_Code'].astype(str)

In [81]:
df['CityTitle'] = df['CityTitle'].astype(str)

In [82]:
features = ['CustomerBk','AreaBk', 'Product_Code','DistPathCode','CustomerTypeBk','CityTitle','OfficeBk','LastCustomerClassBk'] 
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df[features].values, tf.string)
            
        )
    )
)

2022-12-25 17:06:05.670901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-25 17:06:05.672213: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-25 17:06:05.674555: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gljhub): /proc/driver/nvidia/version does not exist
2022-12-25 17:06:05.720867: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [83]:
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(8,), dtype=tf.string, name=None)>

In [84]:
ratings = (dataset
           .map(lambda rating: {'user_id': rating[0],'Area_Code':rating[1],'product_title': rating[2],'SalesDocumentType':rating[3],'Customer_Type':rating[4],'city_name':rating[5],'OfficeBk':rating[6],'ClassBk':rating[7]})
           .cache(tempfile.NamedTemporaryFile().name)
)

In [85]:
df

,CustomerBk,CustomerTypeBk,Product_Code,AreaBk,CityTitle,OfficeBk,DistPathCode,LastCustomerClassBk
0,972442240,37,218,0,ملك شهر,325,0,0
1,959503144,37,275,0,تهران,959,0,300
2,959503144,37,278,0,تهران,959,0,300
3,621004521,37,1511,132,تهران,956,3,100
4,622501396,38,2864,151,تهران,146,2,300
...,...,...,...,...,...,...,...,...
6441006,611040281,37,1080,504,تهران,156,1,100
6441007,954560659,37,1298,0,تهران,250,0,300
6441008,619040248,37,1511,11,تهران,149,1,50
6441009,959503035,37,643,0,تهران,959,0,300


In [86]:
# dataset=[]

In [87]:
# features = ['CustomerCode', 'Product_Code'] 
# dataset = (
#     tf.data.Dataset.from_tensor_slices(
#         (
#             tf.cast(df[features].values, tf.string)
            
#         )
#     )
# )

In [88]:
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(8,), dtype=tf.string, name=None)>

In [89]:
# ratings = (dataset
#            .map(lambda rating: {'user_id': rating[0],'product_title':rating[1]})
#            .cache(tempfile.NamedTemporaryFile().name)
# )

In [90]:
ratings

<CacheDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'Area_Code': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'SalesDocumentType': TensorSpec(shape=(), dtype=tf.string, name=None), 'Customer_Type': TensorSpec(shape=(), dtype=tf.string, name=None), 'city_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'OfficeBk': TensorSpec(shape=(), dtype=tf.string, name=None), 'ClassBk': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [91]:
a = list(df['Product_Code'].unique())
len(a)

543

In [92]:
b = 543*[1]

In [93]:
z=pd.DataFrame({'one':a, 'two':b})

In [94]:
z

,one,two
0,218,1
1,275,1
2,278,1
3,1511,1
4,2864,1
...,...,...
538,1074,1
539,1777,1
540,3160,1
541,1542,1


In [95]:
features = ['one'] 
dataset1 = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(z[features].values, tf.string)
        )
    )
)

In [96]:
products = (dataset1
          .map(lambda x: x[0])
          .cache(tempfile.NamedTemporaryFile().name))

In [97]:
len(df)

6441011

In [98]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(9036061 , seed=42, reshuffle_each_iteration=False)

train = shuffled.take(9036061)
test = shuffled.skip(9036061).take(1)

In [99]:
products

<CacheDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [100]:
df

,CustomerBk,CustomerTypeBk,Product_Code,AreaBk,CityTitle,OfficeBk,DistPathCode,LastCustomerClassBk
0,972442240,37,218,0,ملك شهر,325,0,0
1,959503144,37,275,0,تهران,959,0,300
2,959503144,37,278,0,تهران,959,0,300
3,621004521,37,1511,132,تهران,956,3,100
4,622501396,38,2864,151,تهران,146,2,300
...,...,...,...,...,...,...,...,...
6441006,611040281,37,1080,504,تهران,156,1,100
6441007,954560659,37,1298,0,تهران,250,0,300
6441008,619040248,37,1511,11,تهران,149,1,50
6441009,959503035,37,643,0,تهران,959,0,300


In [101]:
# l11

In [102]:
# user_area

In [ ]:
product_title = products.batch(1_000)
user_ids = ratings.batch(10_000_000).map(lambda x: x["user_id"])
user_area = ratings.batch(10_000_000).map(lambda x: x["Area_Code"])
user_sale = ratings.batch(10_000_000).map(lambda x: x["SalesDocumentType"])
user_types = ratings.batch(10_000_000).map(lambda x: x["Customer_Type"])
user_city = ratings.batch(10_000_000).map(lambda x: x["city_name"])
user_city1 = ratings.batch(10_000_000).map(lambda x: x["OfficeBk"])
user_city2 = ratings.batch(10_000_000).map(lambda x: x["ClassBk"])

unique_product_title = np.unique(np.concatenate(list(product_title)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_area = np.unique(np.concatenate(list(user_area)))
unique_user_sale = np.unique(np.concatenate(list(user_sale)))
unique_user_types = np.unique(np.concatenate(list(user_types)))
unique_user_city = np.unique(np.concatenate(list(user_city)))
unique_user_city1 = np.unique(np.concatenate(list(user_city1)))
unique_user_city2 = np.unique(np.concatenate(list(user_city2)))

unique_product_title[:20]

2022-12-25 17:06:11.143846: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
unique_product_title

In [ ]:
# product_title = products.batch(1_000)
# user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])


# unique_product_title = np.unique(np.concatenate(list(product_title)))
# unique_user_ids = np.unique(np.concatenate(list(user_ids)))


# unique_product_title[:20]

In [ ]:
len(unique_product_title)

In [ ]:
unique_user_ids

In [ ]:
unique_product_title

In [ ]:
embedding_dimension = 320

In [ ]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, 30)
])

In [ ]:
user_model1 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_area, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_area) + 1, 30)
])

In [ ]:
user_model2 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_sale, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_sale) + 1, 30)
])

In [ ]:
user_model3 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_types, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_types) + 1, 30)
])

In [ ]:
user_model4 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_city, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_city) + 1, 30)
])

In [ ]:
user_model5 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_city1, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_city1) + 1, 30)
])

In [ ]:
user_model6 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_city2, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_city2) + 1, 30)
])

In [ ]:
len(unique_user_ids)

In [ ]:
type(user_model)

In [ ]:
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_product_title, mask_token=None),
  tf.keras.layers.Embedding(len(unique_product_title) + 1, 210)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=products.batch(128).map(product_model)
)

In [ ]:
retrieval_task_layer = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
# class MovielensModel(tfrs.models.Model):

#   def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
#     # We take the loss weights in the constructor: this allows us to instantiate
#     # several model objects with different loss weights.

#     super().__init__()

#     embedding_dimension = 32

#     # User and movie models.
#     self.movie_model: tf.keras.Model = movie_model
#     self.user_model: tf.keras.Model = user_model
#     self.user_model1: tf.keras.Model = user_model1
    
#     # A small model to take in user and movie embeddings and predict ratings.
#     # We can make this as complicated as we want as long as we output a scalar
#     # as our prediction.
#     self.rating_model = tf.keras.Sequential([
#         tf.keras.layers.Dense(256, activation="relu"),
#         tf.keras.layers.Dense(128, activation="relu"),
#         tf.keras.layers.Dense(1),
#     ])

#     # The tasks.

#     self.rating_task: tf.keras.layers.Layer = task
#     self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
#         metrics=tfrs.metrics.FactorizedTopK(
#             candidates=movies.batch(128).map(self.movie_model)
#         )
#     )

#     # The loss weights.


#   def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
#     # We pick out the user features and pass them into the user model.
#     user_embeddings = self.user_model(features["user_id"])
#     user_embeddings1 = self.user_model1(features["Customer_Type"])
#     # And pick out the movie features and pass them into the movie model.
#     movie_embeddings = self.movie_model(features["movie_title"])
    
#     return (
#         user_embeddings,
#         user_embeddings1,
#         # We apply the multi-layered rating model to a concatentation of
#         # user and movie embeddings.
#         self.rating_model(
#             tf.concat([user_embeddings, user_embeddings1], axis=1)
#         ),
#     )

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

#     ratings = features.pop("user_rating")

#     user_embeddings, movie_embeddings, rating_predictions = self(features)

#     # We compute the loss for each task.
#     rating_loss = self.rating_task(
#         labels=ratings,
#         predictions=rating_predictions,
#     )
#     retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

#     # And combine them using the loss weights.
#     return (self.rating_weight * rating_loss
#             + self.retrieval_weight * retrieval_loss)

In [ ]:
user_model

In [ ]:
class productModel(tfrs.models.Model):
  """MovieLens candidate generation model"""
 
  def __init__( self,user_model,user_model1,user_model2,user_model3,user_model4,user_model5,user_model6, product_model,retrieval_task_layer):
    
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = user_model
    self.user_model1: tf.keras.Model = user_model1
    self.user_model2: tf.keras.Model = user_model2
    self.user_model3: tf.keras.Model = user_model3
    self.user_model4: tf.keras.Model = user_model4
    self.user_model5: tf.keras.Model = user_model5
    self.user_model6: tf.keras.Model = user_model6
    self.retrieval_task_layer: tf.keras.layers.Layer = retrieval_task_layer


    
    # #self.query_embeddings_ds_1 = tf.keras.layers.Dense(128, activation='relu')
    # self.query_embeddings_ds_2 = tf.keras.layers.Dense(64, activation='relu')
    # self.query_embeddings_ds_3 = tf.keras.layers.Dense(32, activation='relu')
    
    # self.positive_candidate_embeddings_ds_1 = tf.keras.layers.Dense(128, activation='relu')
    # self.positive_candidate_embeddings_ds_2 = tf.keras.layers.Dense(64, activation='relu')
    # self.positive_candidate_embeddings_ds_3 = tf.keras.layers.Dense(32, activation='relu')
    
    # self.prd_sls_dff_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
    # self.cst_sls_dff_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
    # self.int_q_su_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
    # self.prv_int_q_su_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
    
 
 #def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
  def compute_loss(self, features, training=False) -> tf.Tensor:
    
    print(features)
    user_embeddings = self.user_model(features["user_id"])
    user_embeddings1 = self.user_model1(features["Area_Code"])
    user_embeddings2 = self.user_model2(features["SalesDocumentType"])
    user_embeddings3 = self.user_model3(features["Customer_Type"])
    user_embeddings4 = self.user_model4(features["city_name"])
    user_embeddings5 = self.user_model5(features["OfficeBk"])
    user_embeddings6 = self.user_model6(features["ClassBk"])
    product_embeddings = self.product_model(features["product_title"])
    
    query_embeddings_concats = tf.concat([user_embeddings, user_embeddings1,user_embeddings2,user_embeddings3,user_embeddings4,user_embeddings5,user_embeddings6], axis=1)
    
#     query_embeddings_final = self.query_embeddings_ds_3(
#         self.query_embeddings_ds_2(
#             self.query_embeddings_ds_1(
#                 query_embeddings_concats
#             )
#         )
#     )
    
    query_embeddings_final = query_embeddings_concats
    
    
    #t2 = self.res(self.raw_user_age_normalization_layer(features["raw_user_age"]))
    
    positive_candidate_embeddings_concats = product_embeddings
    
    positive_candidate_embeddings_final = positive_candidate_embeddings_concats

#     print(query_embeddings)
#     print(positive_candidate_embeddings)

    loss = self.retrieval_task_layer(
        query_embeddings_final,
        positive_candidate_embeddings_final
        # ,compute_metrics=not training  # To speed up training
    )
    return loss

In [ ]:
# class MovielensModel(tfrs.Model):

#   def __init__(self, user_model, movie_model):
#     super().__init__()
#     self.movie_model: tf.keras.Model = movie_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
#     # We pick out the user features and pass them into the user model.
#     user_embeddings = self.user_model(features["user_id"])
#     # And pick out the movie features and pass them into the movie model,
#     # getting embeddings back.
#     positive_movie_embeddings = self.movie_model(features["movie_title"])

#     # The task computes the loss and the metrics.
#     return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
# class NoBaseClassMovielensModel(tf.keras.Model):

#   def __init__(self, user_model, movie_model):
#     super().__init__()
#     self.movie_model: tf.keras.Model = movie_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Set up a gradient tape to record gradients.
#     with tf.GradientTape() as tape:

#       # Loss computation.
#       user_embeddings = self.user_model(features["user_id"])
#       positive_movie_embeddings = self.movie_model(features["movie_title"])
#       loss = self.task(user_embeddings, positive_movie_embeddings)

#       # Handle regularization losses as well.
#       regularization_loss = sum(self.losses)

#       total_loss = loss + regularization_loss

#     gradients = tape.gradient(total_loss, self.trainable_variables)
#     self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

#   def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Loss computation.
#     user_embeddings = self.user_model(features["user_id"])
#     positive_movie_embeddings = self.movie_model(features["movie_title"])
#     loss = self.task(user_embeddings, positive_movie_embeddings)

#     # Handle regularization losses as well.
#     regularization_loss = sum(self.losses)

#     total_loss = loss + regularization_loss

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

In [ ]:
# class MovielensModel(tfrs.models.Model):
#   """MovieLens candidate generation model"""
 
#   def __init__( self,user_model,product_model,retrieval_task_layer):
    
#     super().__init__()
#     self.product_model: tf.keras.Model = product_model
#     self.user_model: tf.keras.Model = user_model
#     # self.user_model1: tf.keras.Model = user_model1
#     # self.user_model2: tf.keras.Model = user_model2
#     # self.user_model3: tf.keras.Model = user_model3
#     # self.user_model4: tf.keras.Model = user_model4
#     self.retrieval_task_layer: tf.keras.layers.Layer = retrieval_task_layer


    
#     # #self.query_embeddings_ds_1 = tf.keras.layers.Dense(128, activation='relu')
#     # self.query_embeddings_ds_2 = tf.keras.layers.Dense(64, activation='relu')
#     # self.query_embeddings_ds_3 = tf.keras.layers.Dense(32, activation='relu')
    
#     # self.positive_candidate_embeddings_ds_1 = tf.keras.layers.Dense(128, activation='relu')
#     # self.positive_candidate_embeddings_ds_2 = tf.keras.layers.Dense(64, activation='relu')
#     # self.positive_candidate_embeddings_ds_3 = tf.keras.layers.Dense(32, activation='relu')
    
#     # self.prd_sls_dff_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
#     # self.cst_sls_dff_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
#     # self.int_q_su_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
#     # self.prv_int_q_su_reshape = tf.keras.layers.Reshape([1])#keras.layers.Reshape((None,1))
    
 
#  #def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
#   def compute_loss(self, features, training=False) -> tf.Tensor:
    
#     print(features)
#     user_embeddings = self.user_model(features["user_id"])

#     product_embeddings = self.product_model(features["product_title"])
    
#     query_embeddings_concats = user_embeddings
    
# #     query_embeddings_final = self.query_embeddings_ds_3(
# #         self.query_embeddings_ds_2(
# #             self.query_embeddings_ds_1(
# #                 query_embeddings_concats
# #             )
# #         )
# #     )
    
#     query_embeddings_final = query_embeddings_concats
    
    
#     #t2 = self.res(self.raw_user_age_normalization_layer(features["raw_user_age"]))
    
#     positive_candidate_embeddings_concats = product_embeddings
    
#     positive_candidate_embeddings_final = positive_candidate_embeddings_concats

# #     print(query_embeddings)
# #     print(positive_candidate_embeddings)

#     loss = self.retrieval_task_layer(
#         query_embeddings_final,
#         positive_candidate_embeddings_final
#         # ,compute_metrics=not training  # To speed up training
#     )
#     return loss

In [ ]:
len(df)

In [ ]:
model = productModel(user_model,user_model1,user_model2,user_model3,user_model4,user_model5,user_model6, product_model,retrieval_task_layer)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
# model = MovielensModel(user_model, product_model,retrieval_task_layer)
# model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(9036061).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=7)

# خواندن لیست مشتریانی که تا کنون تراکنش داشته اند به همراه 8 فیچر آنها 

In [ ]:
dd = pd.read_csv('cust_list_tran_14010425.csv')

In [ ]:
dd

# خواندن لیست مشتریانی که تا کنون تراکنش نداشته اند به همراه 8 فیچر آنها 

In [ ]:
dg = pd.read_csv('cust_list_noTran_14010425.csv')

In [ ]:
dg

In [ ]:
# len(pd.unique(df['CustomerBk']))

In [ ]:
# df1=df.drop_duplicates('CustomerBk', keep='first')

In [ ]:
df['CityTitle'][0]

In [ ]:
dd=dd[['CustomerBk','CustomerTypeBk','AreaBk','CityTitle','OfficeBk','DistPathCode','LastCustomerClassBk']]

In [ ]:
dg=dg[['CustomerBk','CustomerTypeBk','AreaBk','CityTitle','OfficeBk','DistPathCode','LastCustomerClassBk']]

# تجمیع کل مشتریان

In [ ]:
dr = pd.concat([dd,dg])

In [ ]:
dr

In [ ]:
dr2=dr.reset_index()

In [ ]:
dr3=dr2.iloc[:,1:]

In [ ]:
# bbb=[156,959,972]

In [ ]:
dr3

In [ ]:
# for i in range(len(dr2)):
#     dr2['OfficeBk'][i] = int(dr2['OfficeBk'][i])
    

In [ ]:
type(dr2['OfficeBk'][97892])

In [ ]:
aaa

In [ ]:
dr2

In [ ]:
dr2.to_csv('dr2.csv',index=False)

In [ ]:
dr2 = pd.read_csv('dr2.csv')

In [ ]:
dr2

# جدا کردن مشتریان کلانشهرها

In [ ]:
lll=[]
lll1 = []
for i in range(len(dr2)):
    if dr2['OfficeBk'][i] in aaa:
        lll.append(i)  
    else:
        lll1.append(i)

In [ ]:
dr3=dr2.iloc[lll]

In [ ]:
dr3=dr3.reset_index()

In [ ]:
# dd4=dr3.iloc[:,2:]

In [ ]:
# dd4

In [ ]:
df = dr3[['CustomerBk','CustomerTypeBk','AreaBk','CityTitle','OfficeBk','DistPathCode','LastCustomerClassBk']]

In [ ]:
df

In [ ]:
df['CustomerBk'] = df['CustomerBk'].astype(str)
df['CustomerTypeBk'] = df['CustomerTypeBk'].astype(str)
df['AreaBk'] = df['AreaBk'].astype(str)
df['CityTitle'] = df['CityTitle'].astype(str)
df['OfficeBk'] = df['OfficeBk'].astype(str)
df['DistPathCode'] = df['DistPathCode'].astype(str)
df['LastCustomerClassBk'] = df['LastCustomerClassBk'].astype(str)

In [ ]:
df

In [ ]:
df5=df.iloc[:10000,:]

In [ ]:
df5=df5.reset_index()

In [ ]:
df5=df5.iloc[:,1:]

In [ ]:
df5

In [ ]:
features = ['CustomerBk','AreaBk','DistPathCode','CustomerTypeBk','CityTitle','OfficeBk','LastCustomerClassBk'] 
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df5[features].values, tf.string)
            
        )
    )
)

In [ ]:
ratings = (dataset
           .map(lambda rating: {'user_id': rating[0],'Area_Code':rating[1],'SalesDocumentType':rating[2],'Customer_Type':rating[3],'city_name':rating[4],'OfficeBk':rating[5],'ClassBk':rating[6]})
           .cache(tempfile.NamedTemporaryFile().name)
)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(10000 , seed=42, reshuffle_each_iteration=False)

train = shuffled.take(10000)


In [ ]:
l=[]
for i in range(10000):
    l1=[]
    # index = tfrs.layers.factorized_top_k.BruteForce(model.user_model,100)
    # # recommends movies out of the entire movies dataset.
    # index.index_from_dataset(
    #   tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    # )
    brute_force_layer = tfrs.layers.factorized_top_k.BruteForce(k=150)
    brute_force_layer.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    )

    cached_test = train.batch(1)

    test_sample = cached_test.skip(i).take(1)
    test_sample_map = test_sample.map(
            lambda features: tf.concat([model.user_model(features["user_id"]), model.user_model1(features["Area_Code"]),model.user_model2(features["SalesDocumentType"]),model.user_model3(features["Customer_Type"]), model.user_model4(features["city_name"]), model.user_model5(features["OfficeBk"]), model.user_model6(features["ClassBk"])], axis=1)
        )

    test_sample_map_arr = []
    for item in test_sample_map.take(1).as_numpy_iterator():
        test_sample_map_arr.append(item[0])
    test_sample_map_arr = np.array(test_sample_map_arr)

    afinity_scores, a = brute_force_layer(test_sample_map_arr)

    for item in test_sample.as_numpy_iterator():
        print(f"Recommendations for user {item['user_id']} using BruteForce: {i}")
        # ppp = 1

    l1.append(int(item['user_id']))
    # Get recommendations.
    # _, titles = index(tf.constant([i]))
    z=list(a[0, :150].numpy())
    for j in z:
        l1.append(int(j))
        
    l.append(l1)

In [ ]:
len(l)

In [ ]:
dd1=pd.DataFrame(l)

In [ ]:
dd1

In [ ]:
dd1.to_csv('part1.csv')

In [ ]:
df5=df.iloc[10000:20000,:]

In [ ]:
df5=df5.reset_index()

In [ ]:
df5=df5.iloc[:,1:]

In [ ]:
df5

In [ ]:
features = ['CustomerBk','AreaBk','DistPathCode','CustomerTypeBk','CityTitle','OfficeBk','LastCustomerClassBk'] 
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df5[features].values, tf.string)
            
        )
    )
)

In [ ]:
ratings = (dataset
           .map(lambda rating: {'user_id': rating[0],'Area_Code':rating[1],'SalesDocumentType':rating[2],'Customer_Type':rating[3],'city_name':rating[4],'OfficeBk':rating[5],'ClassBk':rating[6]})
           .cache(tempfile.NamedTemporaryFile().name)
)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(10000 , seed=42, reshuffle_each_iteration=False)

train = shuffled.take(10000)


In [ ]:
l=[]
for i in range(10000):
    l1=[]
    # index = tfrs.layers.factorized_top_k.BruteForce(model.user_model,100)
    # # recommends movies out of the entire movies dataset.
    # index.index_from_dataset(
    #   tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    # )
    brute_force_layer = tfrs.layers.factorized_top_k.BruteForce(k=150)
    brute_force_layer.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    )

    cached_test = train.batch(1)

    test_sample = cached_test.skip(i).take(1)
    test_sample_map = test_sample.map(
            lambda features: tf.concat([model.user_model(features["user_id"]), model.user_model1(features["Area_Code"]),model.user_model2(features["SalesDocumentType"]),model.user_model3(features["Customer_Type"]), model.user_model4(features["city_name"]), model.user_model5(features["OfficeBk"]), model.user_model6(features["ClassBk"])], axis=1)
        )

    test_sample_map_arr = []
    for item in test_sample_map.take(1).as_numpy_iterator():
        test_sample_map_arr.append(item[0])
    test_sample_map_arr = np.array(test_sample_map_arr)

    afinity_scores, a = brute_force_layer(test_sample_map_arr)

    for item in test_sample.as_numpy_iterator():
        print(f"Recommendations for user {item['user_id']} using BruteForce: {i}")
        # ppp = 1

    l1.append(int(item['user_id']))
    # Get recommendations.
    # _, titles = index(tf.constant([i]))
    z=list(a[0, :150].numpy())
    for j in z:
        l1.append(int(j))
        
    l.append(l1)

In [ ]:
len(l)

In [ ]:
dd1=pd.DataFrame(l)

In [ ]:
dd1

In [ ]:
dd1.to_csv('part2.csv')

In [ ]:
df5=df.iloc[20000:30000,:]

In [ ]:
df5=df5.reset_index()

In [ ]:
df5=df5.iloc[:,1:]

In [ ]:
df5

In [ ]:
features = ['CustomerBk','AreaBk','DistPathCode','CustomerTypeBk','CityTitle','OfficeBk','LastCustomerClassBk'] 
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df5[features].values, tf.string)
            
        )
    )
)

In [ ]:
ratings = (dataset
           .map(lambda rating: {'user_id': rating[0],'Area_Code':rating[1],'SalesDocumentType':rating[2],'Customer_Type':rating[3],'city_name':rating[4],'OfficeBk':rating[5],'ClassBk':rating[6]})
           .cache(tempfile.NamedTemporaryFile().name)
)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(10000 , seed=42, reshuffle_each_iteration=False)

train = shuffled.take(10000)


In [ ]:
l=[]
for i in range(10000):
    l1=[]
    # index = tfrs.layers.factorized_top_k.BruteForce(model.user_model,100)
    # # recommends movies out of the entire movies dataset.
    # index.index_from_dataset(
    #   tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    # )
    brute_force_layer = tfrs.layers.factorized_top_k.BruteForce(k=150)
    brute_force_layer.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    )

    cached_test = train.batch(1)

    test_sample = cached_test.skip(i).take(1)
    test_sample_map = test_sample.map(
            lambda features: tf.concat([model.user_model(features["user_id"]), model.user_model1(features["Area_Code"]),model.user_model2(features["SalesDocumentType"]),model.user_model3(features["Customer_Type"]), model.user_model4(features["city_name"]), model.user_model5(features["OfficeBk"]), model.user_model6(features["ClassBk"])], axis=1)
        )

    test_sample_map_arr = []
    for item in test_sample_map.take(1).as_numpy_iterator():
        test_sample_map_arr.append(item[0])
    test_sample_map_arr = np.array(test_sample_map_arr)

    afinity_scores, a = brute_force_layer(test_sample_map_arr)

    for item in test_sample.as_numpy_iterator():
        print(f"Recommendations for user {item['user_id']} using BruteForce: {i}")
        # ppp = 1

    l1.append(int(item['user_id']))
    # Get recommendations.
    # _, titles = index(tf.constant([i]))
    z=list(a[0, :150].numpy())
    for j in z:
        l1.append(int(j))
        
    l.append(l1)

In [ ]:
len(l)

In [ ]:
dd1=pd.DataFrame(l)

In [ ]:
dd1

In [ ]:
dd1.to_csv('part3.csv')

In [ ]:
df5=df.iloc[30000:40000,:]

In [ ]:
df5=df5.reset_index()

In [ ]:
df5=df5.iloc[:,1:]

In [ ]:
df5

In [ ]:
features = ['CustomerBk','AreaBk','DistPathCode','CustomerTypeBk','CityTitle','OfficeBk','LastCustomerClassBk'] 
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df5[features].values, tf.string)
            
        )
    )
)

In [ ]:
ratings = (dataset
           .map(lambda rating: {'user_id': rating[0],'Area_Code':rating[1],'SalesDocumentType':rating[2],'Customer_Type':rating[3],'city_name':rating[4],'OfficeBk':rating[5],'ClassBk':rating[6]})
           .cache(tempfile.NamedTemporaryFile().name)
)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(10000 , seed=42, reshuffle_each_iteration=False)

train = shuffled.take(10000)


In [ ]:
l=[]
for i in range(10000):
    l1=[]
    # index = tfrs.layers.factorized_top_k.BruteForce(model.user_model,100)
    # # recommends movies out of the entire movies dataset.
    # index.index_from_dataset(
    #   tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    # )
    brute_force_layer = tfrs.layers.factorized_top_k.BruteForce(k=150)
    brute_force_layer.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    )

    cached_test = train.batch(1)

    test_sample = cached_test.skip(i).take(1)
    test_sample_map = test_sample.map(
            lambda features: tf.concat([model.user_model(features["user_id"]), model.user_model1(features["Area_Code"]),model.user_model2(features["SalesDocumentType"]),model.user_model3(features["Customer_Type"]), model.user_model4(features["city_name"]), model.user_model5(features["OfficeBk"]), model.user_model6(features["ClassBk"])], axis=1)
        )

    test_sample_map_arr = []
    for item in test_sample_map.take(1).as_numpy_iterator():
        test_sample_map_arr.append(item[0])
    test_sample_map_arr = np.array(test_sample_map_arr)

    afinity_scores, a = brute_force_layer(test_sample_map_arr)

    for item in test_sample.as_numpy_iterator():
        print(f"Recommendations for user {item['user_id']} using BruteForce: {i}")
        # ppp = 1

    l1.append(int(item['user_id']))
    # Get recommendations.
    # _, titles = index(tf.constant([i]))
    z=list(a[0, :150].numpy())
    for j in z:
        l1.append(int(j))
        
    l.append(l1)

In [ ]:
len(l)

In [ ]:
dd1=pd.DataFrame(l)

In [ ]:
dd1

In [ ]:
dd1.to_csv('part4.csv')

In [ ]:
df5=df.iloc[40000:,:]

In [ ]:
df5=df5.reset_index()

In [ ]:
df5=df5.iloc[:,1:]

In [ ]:
df5

In [ ]:
features = ['CustomerBk','AreaBk','DistPathCode','CustomerTypeBk','CityTitle','OfficeBk','LastCustomerClassBk'] 
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df5[features].values, tf.string)
            
        )
    )
)

In [ ]:
ratings = (dataset
           .map(lambda rating: {'user_id': rating[0],'Area_Code':rating[1],'SalesDocumentType':rating[2],'Customer_Type':rating[3],'city_name':rating[4],'OfficeBk':rating[5],'ClassBk':rating[6]})
           .cache(tempfile.NamedTemporaryFile().name)
)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(11008 , seed=42, reshuffle_each_iteration=False)

train = shuffled.take(11008)


In [ ]:
l=[]
for i in range(11008):
    l1=[]
    # index = tfrs.layers.factorized_top_k.BruteForce(model.user_model,100)
    # # recommends movies out of the entire movies dataset.
    # index.index_from_dataset(
    #   tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    # )
    brute_force_layer = tfrs.layers.factorized_top_k.BruteForce(k=150)
    brute_force_layer.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(product_model)))
    )

    cached_test = train.batch(1)

    test_sample = cached_test.skip(i).take(1)
    test_sample_map = test_sample.map(
            lambda features: tf.concat([model.user_model(features["user_id"]), model.user_model1(features["Area_Code"]),model.user_model2(features["SalesDocumentType"]),model.user_model3(features["Customer_Type"]), model.user_model4(features["city_name"]), model.user_model5(features["OfficeBk"]), model.user_model6(features["ClassBk"])], axis=1)
        )

    test_sample_map_arr = []
    for item in test_sample_map.take(1).as_numpy_iterator():
        test_sample_map_arr.append(item[0])
    test_sample_map_arr = np.array(test_sample_map_arr)

    afinity_scores, a = brute_force_layer(test_sample_map_arr)

    for item in test_sample.as_numpy_iterator():
        print(f"Recommendations for user {item['user_id']} using BruteForce: {i}")
        # ppp = 1

    l1.append(int(item['user_id']))
    # Get recommendations.
    # _, titles = index(tf.constant([i]))
    z=list(a[0, :150].numpy())
    for j in z:
        l1.append(int(j))
        
    l.append(l1)

In [ ]:
len(l)

In [ ]:
dd1=pd.DataFrame(l)

In [ ]:
dd1

In [ ]:
dd1.to_csv('part3.csv')

In [ ]:
dff1 = pd.read_csv('part1.csv')

In [ ]:
dff2 = pd.read_csv('part2.csv')

In [ ]:
dff3 = pd.read_csv('part3.csv')

In [ ]:
dff4 = pd.read_csv('part4.csv')

In [ ]:
dff5 = pd.read_csv('part5.csv')

In [ ]:
dff5=pd.concat([dff1, dff2,dff3,dff4,dff5])
dff5=dff5.reset_index()
dff5=dff5.iloc[:,1:]


In [ ]:
dff5

In [ ]:
dff5=dff5.iloc[:,1:]

In [ ]:
dff5

In [ ]:
dff5.to_csv('kalan_final.csv')